<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1">Load Data</a></span></li><li><span><a href="#Train-Test-Split" data-toc-modified-id="Train-Test-Split-2">Train Test Split</a></span></li><li><span><a href="#Baseline-Model" data-toc-modified-id="Baseline-Model-3">Baseline Model</a></span></li><li><span><a href="#Train-Models" data-toc-modified-id="Train-Models-4">Train Models</a></span></li><li><span><a href="#Model-Comparison" data-toc-modified-id="Model-Comparison-5">Model Comparison</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-6">Results</a></span></li></ul></div>

In [48]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
plt.style.use('ggplot')

# Load Data

In [14]:
data = pd.read_csv('Data/processed_data')
data.head(10)

,Unnamed: 0,index,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,...,MW,LogP,NumHDonors,NumHAcceptors,rotatable_bonds,number_of_atoms,molar_refractivity,SA_mapping,heavy_atoms,num_of_rings
0,0,0,CHEMBL231779,APIXABAN,4,459.51,0,2.70,Apixaban,COc1ccc(-n2nc(C(N)=O)c3c2C(=O)N(c2ccc(N4CCCCC4...,...,459.506,2.69960,1,6,5,34,126.6604,110.76,34,5
1,1,1,CHEMBL186,CEFEPIME,4,480.57,0,-1.28,cefepime,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)[O-])=C(C[N...,...,480.572,-1.27990,2,10,7,32,117.1475,150.04,32,4
2,2,2,CHEMBL240597,CHENODIOL,4,392.58,0,4.48,chenodeoxycholic-acid,C[C@H](CCC(=O)O)[C@H]1CC[C@H]2[C@H]3[C@H](CC[C...,...,392.580,4.47790,3,3,4,28,108.6474,77.76,28,4
3,3,3,CHEMBL1098,BUPIVACAINE,4,288.44,0,3.90,bupivacaine,CCCCN1CCCCC1C(=O)Nc1c(C)cccc1C,...,288.435,3.89654,1,2,5,21,88.6657,32.34,21,2
4,4,4,CHEMBL345714,ACIPIMOX,4,154.12,0,-0.28,Acipimox,Cc1cnc(C(=O)O)c[n+]1[O-],...,154.125,-0.27838,1,3,1,11,34.8933,77.13,11,1
5,5,5,CHEMBL498,CHLORPROPAMIDE,4,276.75,0,1.74,Chlorpropamide,CCCNC(=O)NS(=O)(=O)c1ccc(Cl)cc1,...,276.745,1.73790,2,3,4,17,65.4632,75.27,17,1
6,6,6,CHEMBL475534,NITRENDIPINE,4,360.37,0,2.57,nitrendipine,CCOC(=O)C1=C(C)NC(C)=C(C(=O)OC)C1c1cccc([N+](=...,...,360.366,2.56570,1,7,5,26,93.0221,107.77,26,2
7,7,7,CHEMBL1625,OXYBENZONE,4,228.25,0,2.63,Oxybenzone,COc1ccc(C(=O)c2ccccc2)c(O)c1,...,228.247,2.63180,1,3,3,17,64.5333,46.53,17,2
8,8,8,CHEMBL2359059,NaN,0,403.95,0,3.99,Propiverine hydrochloride,CCCOC(C(=O)OC1CCN(C)CC1)(c1ccccc1)c1ccccc1.Cl,...,403.950,4.41600,0,4,7,28,113.7660,38.77,28,3
9,9,9,CHEMBL421,SULFASALAZINE,4,398.40,0,3.70,Sulfasalazine,O=C(O)c1cc(/N=N/c2ccc(S(=O)(=O)Nc3ccccn3)cc2)c...,...,398.400,3.70160,3,7,6,28,100.7296,141.31,28,3


In [15]:
data.columns

Index(['Unnamed: 0', 'index', 'Molecule ChEMBL ID', 'Molecule Name',
       'Molecule Max Phase', 'Molecular Weight', '#RO5 Violations', 'AlogP',
       'Compound Key', 'Smiles', 'Standard Type', 'Standard Relation',
       'Standard Value', 'Standard Units', 'MW', 'LogP', 'NumHDonors',
       'NumHAcceptors', 'rotatable_bonds', 'number_of_atoms',
       'molar_refractivity', 'SA_mapping', 'heavy_atoms', 'num_of_rings'],
      dtype='object')

# Train Test Split

In [16]:
predictors = ['rotatable_bonds', 'number_of_atoms', 'SA_mapping','num_of_rings', 'MW', 'LogP', 'NumHDonors', 'NumHAcceptors']
target = 'Standard Value'

# Subset drugs above candidate threshold 
data['above_threshold'] = data[target] > 0.6
data['above_threshold'] = data['above_threshold'].astype(int)

X_train, X_test, y_train,  y_test = train_test_split(data, data[target], test_size = 0.2, stratify = data['above_threshold'])

# Baseline Model

In [46]:
# Fit linear regression model on training data
baseline_model = LinearRegression().fit(X_train[predictors], y_train)

In [53]:
baseline_model.score(X_train[predictors], y_train)

0.012600172719526737

In [59]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    """
    Calculates performance of model on train and test sets
    Parameters
    ----------
    model: fitted sklern model
    X_train, X_test, y_train, y_test : data
    Returns
    -------
    (train_score, test_score) : scores on the training and test sets
    """
    return mean_squared_error(y_train, model.predict(X_train[predictors])), mean_squared_error(y_test, model.predict(X_test[predictors]))
    

In [60]:
evaluate_model(baseline_model, X_train, X_test, y_train, y_test)

(0.01895542893035431, 0.017886115912702425)

# Train Models

In [62]:
from sklearn.ensemble import RandomForestRegressor

In [80]:
rf_model = RandomForestRegressor(n_estimators=100, max_depth= 8)

In [81]:
rf_model.fit(X_train[predictors], y_train)

RandomForestRegressor(max_depth=8)

In [82]:
evaluate_model(rf_model, X_train, X_test, y_train, y_test)

(0.01107069807153664, 0.01807669117784998)

In [87]:
rf_model.predict(data[data['Molecule Name'] == 'REMDESIVIR'][predictors])

array([0.50752936, 0.50752936])

In [88]:
data[data['Molecule Name'] == 'REMDESIVIR'][target]

251     0.5934
1177    0.9631
Name: Standard Value, dtype: float64

# Model Comparison

# Results